# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [2]:
import numpy as np
import pandas as pd

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [3]:
df_data = pd.read_csv('Data\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('Data\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)

df_data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\Voorbeeld_clusterdata.txt'

## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [4]:
class KMCA:
    def __init__(self, k=6, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(K):
        self.k = K

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        def pred(self, data):
            distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            return classification
        
        self.pred_classifications = list()
        
        if np.isfortran(data):
            for element in data:
                self.pred_classifications.append(pred(self, element))
        else:
            self.pred_classifications.append(pred(self, data))
            
        return self.pred_classifications

### Own cluster method:
#### Quantile assignment
De namen van de gebruikte kwantielen worden gedefinieerd in de lijst 'quantiles'. De 'pandas.qcut' functie wordt uitgevoerd op elk van de r-dimensies (kolommen van df_data). Deze functie neemt van een kolom de waarden van alle genen (rijen) en berekent aan de hand daarvan in welk kwantiel elke waarde hoort. De hoeveelheid kwantielen die worden gebruikt hangt af van de lengte van de lijst 'quantiles', zodat er meer kwantielen worden gevormd wanneer andere namen worden toegevoegd aan de lijst.

De rijen code die gevolgd worden door #test zijn niet belangrijk voor het programma, maar geven een inzicht in de berekening van kwantielen voor alle r-dimensies.

In [5]:
def assign_quantiles(data,quantiles):
    leveled = data.copy()
    for rX in range(1,len(leveled.columns)+1):
        print('r'+str(rX)) #test
        for q_nr in range(len(quantiles)): #test
            print('\t', quantiles[q_nr], leveled[rX].quantile(q=(q_nr+1)*(1/len(quantiles)))) #test
        leveled[rX] = pd.qcut(leveled[rX], len(quantiles), labels=quantiles)
    return leveled

quantiles = ['1st','2nd','3rd']
assign_quantiles(df_data,quantiles).head(10)

NameError: name 'df_data' is not defined

## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [4]:
kmca = KMCA()
kmca.fit(df_data.values)
df_clusterd['KMCA'] = kmca.predict(df_data.values)

df_clusterd.head()

,KMCA
0,
846160,0
820434,4
849103,2
846353,3
848613,2


### save results:

In [12]:
df_clusterd.to_csv(r'Data\clusterresultaten.txt', header=None, index=True, sep=' ')